In [13]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Activation, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.callbacks import TensorBoard
import keras

## training params

In [14]:

batch_size = 128
nb_epoch = 100

pool_size = (2, 2)                    # size of poolin area for max pooling
prob_drop_conv = 0.2                  # drop probability for dropout @ conv layer
prob_drop_hidden = 0.5                # drop probability for dropout @ fc layer

## data params

In [15]:
# data params
nb_classes = 10


## Load MNIST dataset

In [16]:
(X_train_raw, Y_train_raw), (X_test_raw, Y_test_raw) = mnist.load_data()
print('X_train_raw original shape:', X_train_raw.shape)

X_train_raw original shape: (60000, 28, 28)


### Reshape Data from image matrix to array & normalize


In [17]:
if K.image_dim_ordering() == 'th':

    # Theano
    X_train = X_train_raw.reshape(X_train_raw.shape[0], 1, X_train_raw.shape[1], X_train_raw.shape[2]).astype('float32')
    X_train /= 255.

    X_test = X_test_raw.reshape(X_test_raw.shape[0], 1, X_test_raw.shape[1], X_test_raw.shape[2]).astype('float32')
    X_test /= 255.
    
    input_shape = (1, X_train_raw.shape[1], X_train_raw.shape[2])
    
else:
    
    # TensorFlow
    X_train = X_train_raw.reshape(X_train_raw.shape[0], X_train_raw.shape[1], X_train_raw.shape[2], 1).astype('float32')
    X_train /= 255.

    X_test = X_test_raw.reshape(X_test_raw.shape[0], X_test_raw.shape[1], X_test_raw.shape[2], 1).astype('float32')
    X_test /= 255.
    
    input_shape = (X_train_raw.shape[1], X_train_raw.shape[2], 1)
    
print('X_train shape: ', X_train.shape)
print('X_test shape: ', X_test.shape)


X_train shape:  (60000, 28, 28, 1)
X_test shape:  (10000, 28, 28, 1)


In [18]:
Y_train = np_utils.to_categorical(Y_train_raw)

Y_test = np_utils.to_categorical(Y_test_raw)

## Convolutional Network model

In [19]:
def init_weights(shape, name=None):
    return keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)

In [23]:
model = Sequential()

# conv1 layer
model.add(Convolution2D(32, 3, 3, border_mode='same', input_shape=input_shape, init=init_weights))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Convolution2D(64, 3, 3, border_mode='same', init=init_weights))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Convolution2D(128, 3, 3, border_mode='same', init=init_weights))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
# why is it 625 here??
model.add(Dense(output_dim=625, init=init_weights))
model.add(Activation('relu'))
model.add(Dropout(prob_drop_hidden))

# fc2 layer
model.add(Dense(output_dim=10, init=init_weights))
model.add(Activation('softmax'))
          
model.compile(optimizer=RMSprop(lr=0.001, rho=0.9), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

/home/ubuntu/anaconda2/envs/tensorflow-py3/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(28, 28, 1..., padding="same", kernel_initializer=<function ...)`


TypeError: __call__() missing 1 required positional argument: 'shape'

## Train

In [10]:
history = model.fit(X_train, Y_train, nb_epoch=nb_epoch, batch_size=batch_size, shuffle=True, validation_split=0.15, verbose=1, callbacks=[TensorBoard(log_dir='./logs/mnist_tensorboard', histogram_freq=1)])

Train on 51000 samples, validate on 9000 samples
Epoch 1/100
51000/51000 [==============================] - 408s - loss: 0.4476 - acc: 0.8472 - val_loss: 0.0845 - val_acc: 0.9726
Epoch 2/100
51000/51000 [==============================] - 453s - loss: 0.1150 - acc: 0.9644 - val_loss: 0.0581 - val_acc: 0.9829
Epoch 3/100
 4864/51000 [=>............................] - ETA: 299s - loss: 0.0853 - acc: 0.9749

KeyboardInterrupt: 

In [ ]:
history.history

## Evaluate

In [ ]:
evaluation = model.evaluate(X_test, Y_test, batch_size=2*batch_size, verbose=1)
print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

## Predict

In [ ]:
test_predictions = model.predict(X_test)
test_prediction_ids = test_predictions.argmax(axis=1)

## Plot some predictions

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
def plots(ims, figsize=(12,6), rows=1, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    for i in range(len(ims)):
        sp = f.add_subplot(rows, len(ims)//rows, i+1)
        if titles is not None:
            sp.set_title(titles[i], fontsize=6)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [ ]:
n_view = 4

idx = np.random.permutation(range(0,len(X_test)))[:n_view]

imgs = X_test[idx,:].reshape(n_view, X_train_raw.shape[1], X_train_raw.shape[2])
titles = test_prediction_ids[idx]

In [ ]:
plots(imgs, titles=titles)
plt.show()

## Examine wrong predictions


In [ ]:
# convert category vector into category label
Y_test_ids = Y_test.argmax(axis=1)

# compare the predicted and actual category labels to find error predictions
errors = np.array(Y_test_ids) - np.array(test_prediction_ids)

# gather all the wrong prediction indices
errors_idx = np.nonzero(errors)[0]

In [ ]:
n_view = 4

random_idx = np.random.permutation(range(0,len(errors_idx)))[:n_view]

# grab a sampling of error indices 
random_errors_idx = errors_idx[random_idx]

# and prepare their images and labels
imgs = X_test[random_errors_idx,:].reshape(n_view, X_train_raw.shape[1], X_train_raw.shape[2])
titles = test_prediction_ids[random_errors_idx]

In [ ]:
plots(imgs, titles=titles)
plt.show()

In [ ]:
len(errors_idx) / 10000.
